In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from files import ImageFile
from processor import (GrayScaleProcessor, BlurProcessor, SobelProcessor, 
                       ThresholdProcessor, MorphologyProcessor, HistogramEqualizationProcessor,
                      CannyProcessor, ScaleProcessor, HoughLinesProcessor,
                      SaveImageProcessor, InvertProcessor, RetrieveImageProcessor)
from segment_processor import (HierarchyContourSegmenter, LargeSegmentFilter, 
                               SmallSegmentFilter, LargeAreaSegmentFilter, ContainedSegmentFilter)
from feature_extractor import (SimpleFeatureExtractor)
from processor_stack import ProcessorStack
import cv2
digitDetectorStack = [ScaleProcessor(maxHeight=800, maxWidth=800),
         SaveImageProcessor(),
         GrayScaleProcessor(),
         BlurProcessor(blur_x=5, blur_y=5),
         HistogramEqualizationProcessor(HistogramEqualizationProcessor.CLAHE),
         #InvertProcessor(),
         #SobelProcessor(xorder=1, yorder=0,ksize=3),
         ThresholdProcessor(ThresholdProcessor.ADAPTIVE, adaptiveBlockSize=31), # 31 for normal. Change this to catch digits!
         #CannyProcessor(100, 200),
         MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(3, 3), op = MorphologyProcessor.ERODE), #Change this
         MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(3, 3), op = MorphologyProcessor.DILATE),
         #HoughLinesProcessor(),
         HierarchyContourSegmenter(),
         RetrieveImageProcessor(),
         LargeSegmentFilter(min_width=0.005, min_height=0.02),
         SmallSegmentFilter(max_width=0.15, max_height=0.4),
         #LargeAreaSegmentFilter(min_area=0.01),
         #ContainedSegmentFilter(),
         SimpleFeatureExtractor(feature_size=32, method = SimpleFeatureExtractor.PAD_BACKGROUND),
         ] #The best 15/12/2015
digitDetector = ProcessorStack(digitDetectorStack)
#test_image= ImageFile('PlateCar')
#test_image= ImageFile('Brad meter 1')
#test_image= ImageFile('Brad meter 2')
#test_image= ImageFile('Davide take 1')
#test_image= ImageFile('Davide take 2')
#test_image= ImageFile('Erik') #Hard set adaptiveBlockSize=9 to catch digits
#test_image= ImageFile('Gro') #Impossible
#test_image= ImageFile('Hans Martin') #quite Hard
#test_image= ImageFile('Knut G')
#test_image= ImageFile('Ole auto')
#test_image= ImageFile('Ole blitz ikke')
#test_image= ImageFile('Ole blitz')
#test_image= ImageFile('Ole liggende')
#test_image= ImageFile('Roy') #small picture, need to scale
#test_image= ImageFile('Sven Erik close up')
#test_image= ImageFile('Sven Erik')
#test_image= ImageFile('Thomas') #Digital
#test_image= ImageFile('Thomas') #Digital
test_image= ImageFile('Heidi') #Digital




digitDetector.process(test_image.image)
digitDetector.display()

showing after ScaleProcessor (waiting for input)
showing after GrayScaleProcessor (waiting for input)
showing after BlurProcessor (waiting for input)
showing after HistogramEqualizationProcessor (waiting for input)
showing after ThresholdProcessor (waiting for input)
showing after MorphologyProcessor (waiting for input)
showing after MorphologyProcessor (waiting for input)
showing segments filtered by HierarchyContourSegmenter (waiting for input)
showing segments filtered by LargeSegmentFilter (waiting for input)
showing segments filtered by SmallSegmentFilter (waiting for input)
Finish displaying ProcessorStack result, waiting for input


In [ ]:
import tesseract
api = tesseract.TessBaseAPI()
api.SetOutputName("outputName");
api.Init(".","eng",tesseract.OEM_DEFAULT)
api.SetPageSegMode(tesseract.PSM_AUTO)
mImgFile = "data/digits1.png"
pixImage=tesseract.pixRead(mImgFile)
api.SetImage(pixImage)
outText=api.GetUTF8Text()
print("OCR output:\n%s"%outText);
api.End()


In [ ]:
import tesseract
api = tesseract.TessBaseAPI()
api.Init(".","eng",tesseract.OEM_DEFAULT)
api.SetVariable("tessedit_char_whitelist", "0123456789abcdefghijklmnopqrstuvwxyz")
api.SetPageSegMode(tesseract.PSM_AUTO)

mImgFile = "data/digits1.png"
mBuffer=open(mImgFile,"rb").read()
result = tesseract.ProcessPagesBuffer(mBuffer,len(mBuffer),api)
print "result(ProcessPagesBuffer)=",result
api.End()


In [ ]:
import tesseract
import cv2
import cv2.cv as cv

image0=cv2.imread("data/digits1.png")
#### you may need to thicken the border in order to make tesseract feel happy to ocr your image #####
offset=20
height,width,channel = image0.shape
image1=cv2.copyMakeBorder(image0,offset,offset,offset,offset,cv2.BORDER_CONSTANT,value=(255,255,255)) 
cv2.namedWindow("Test")
cv2.imshow("Test", image1)
cv2.waitKey(0)
cv2.destroyWindow("Test")
#####################################################################################################
api = tesseract.TessBaseAPI()
api.Init(".","eng",tesseract.OEM_DEFAULT)
api.SetPageSegMode(tesseract.PSM_AUTO)
height1,width1,channel1=image1.shape
print image1.shape
print image1.dtype.itemsize
width_step = width*image1.dtype.itemsize
print width_step
#method 1 
iplimage = cv.CreateImageHeader((width1,height1), cv.IPL_DEPTH_8U, channel1)
cv.SetData(iplimage, image1.tostring(),image1.dtype.itemsize * channel1 * (width1))
tesseract.SetCvImage(iplimage,api)

text=api.GetUTF8Text()
conf=api.MeanTextConf()
image=None
print "..............."
print "Ocred Text: %s"%text
print "Cofidence Level: %d %%"%conf

#method 2:
#cvmat_image=cv.fromarray(image1)
#iplimage =cv.GetImage(cvmat_image)
#print iplimage

#tesseract.SetCvImage(iplimage,api)
#api.SetImage(m_any,width,height,channel1)
#text=api.GetUTF8Text()
#conf=api.MeanTextConf()
#image=None
#print "..............."
#print "Ocred Text: %s"%text
#print "Cofidence Level: %d %%"%conf
api.End()


In [7]:
import numpy
segments = numpy.asarray([[0, 0, 10, 10], [0, 0, 10, 8], [0, 0, 13, 7], [2, 0, 11, 9]])
x1,y1= segments[:,0], segments[:,1]
x2,y2= x1+segments[:,2], y1+segments[:,3]
s = segments[:, 2] * segments[:, 3]
n=len(segments)

x1.shape = x2.shape = y1.shape = y2.shape = s.shape = (n, 1)
X1 = numpy.repeat(x1, n, 1)
X2 = numpy.repeat(x2, n, 1)
Y1 = numpy.repeat(y1, n, 1)
Y2 = numpy.repeat(y2, n, 1)
S  = numpy.repeat(s , n, 1)

#overlapping area
SI= (numpy.maximum(0, numpy.minimum(X2, X2.T) - numpy.maximum(X1, X1.T) ) *
     numpy.maximum(0, numpy.minimum(Y2, Y2.T) - numpy.maximum(Y1, Y1.T)))
#union area
SU = S + S.T - SI
#ratio area
ratio = SI/SU.astype(float)

print (ratio)
#remove ratio which compare a segment with a bigger segment
ratio[S>S.T] = 0
print (ratio)
#deal with equal area segments
print(S)
ST0 = numpy.tril(S.T)
print (ST0)
ratio[S==ST0] = 0
print (ratio)
ratio[ratio>=0.8] = 1
print (ratio)
uniques = numpy.logical_not (numpy.amax(ratio, axis = 1) == 1)
print (numpy.amax(ratio, axis = 1))
print (uniques)

[[ 1.          0.8         0.5785124   0.56692913]
 [ 0.8         1.          0.69306931  0.55652174]
 [ 0.5785124   0.69306931  1.          0.68141593]
 [ 0.56692913  0.55652174  0.68141593  1.        ]]
[[ 1.          0.          0.          0.        ]
 [ 0.8         1.          0.69306931  0.55652174]
 [ 0.5785124   0.          1.          0.68141593]
 [ 0.56692913  0.          0.          1.        ]]
[[100 100 100 100]
 [ 80  80  80  80]
 [ 91  91  91  91]
 [ 99  99  99  99]]
[[100   0   0   0]
 [100  80   0   0]
 [100  80  91   0]
 [100  80  91  99]]
[[ 0.          0.          0.          0.        ]
 [ 0.8         0.          0.69306931  0.55652174]
 [ 0.5785124   0.          0.          0.68141593]
 [ 0.56692913  0.          0.          0.        ]]
[[ 0.          0.          0.          0.        ]
 [ 1.          0.          0.69306931  0.55652174]
 [ 0.5785124   0.          0.          0.68141593]
 [ 0.56692913  0.          0.          0.        ]]
[ 0.          1.          

In [10]:
a = numpy.asarray([1,2,3,4,5,6])
a[a>4] = 0
print a

[1 2 3 4 0 0]


In [19]:
a = numpy.empty(10, dtype = int)
a.fill(10)
print a

[ 10.  10.  10.  10.  10.  10.  10.  10.  10.  10.]


In [15]:
import numpy
a = numpy.asarray([[1,5,3],[4,3,6]])
b = numpy.asarray([[6,2,2],[1,3,4]])
x =  ((a>4) & (b<4)) 
print numpy.nonzero(x)

(array([0], dtype=int64), array([1], dtype=int64))


In [6]:
a = 11/5
print a

2


In [12]:
a = slice(1, 10)

TypeError: unsupported operand type(s) for -: 'slice' and 'int'